In [ ]:
import ipywidgets.widgets as widgets
import sys
sys.path.append('/home/ballsbot/projects/ballsbot/python/lib')
from ballsbot.servos import PCA9685, map_range, get_controls
from ballsbot.utils import keep_rps, run_as_thread
from ballsbot.manipulator import float_map_range, STEP, RPS, IGNORE_LIMIT

In [ ]:
controller = widgets.Controller(index=0)  # http://html5gamepad.com
display(controller)

In [ ]:
car_controls = get_controls()

In [ ]:
servo_config = {
    "channel": 5,
    "min_pulse": 101,
    "max_pulse": 680,
    "default_position": 0.5,
}

servo = PCA9685(servo_config['channel'])

In [ ]:
current_position = servo_config['default_position']  # FIXME
pulse = None

def link_servo_to_controller(servo, controller, axis, servo_config):
    global current_position  # FIXME
    global pulse  # FIXME
    
    ts = None
    while True:
        ts = keep_rps(ts, fps=RPS)

        if len(controller.axes):
            value = controller.axes[axis].value

            if abs(value) > IGNORE_LIMIT:
                increment = float_map_range(value, -1, 1, -STEP, STEP)
                current_position += increment
                if current_position < -1.:
                    current_position = -1.
                elif current_position > 1.:
                    current_position = 1.

        pulse = map_range(
            current_position,
            -1., 1.,
            servo_config['min_pulse'], servo_config['max_pulse']
        )
        servo.set_pulse(pulse)

In [ ]:
run_as_thread(
    link_servo_to_controller,
    servo=servo,
    controller=controller,
    axis=0,
    servo_config=servo_config
)

In [ ]:
print(pulse)
print(current_position)